# Mini-Projet 4 #

### 1.Étude du problème d'otpimisation ###

1. Soit $$f : z \mapsto \frac{1}{2} || z - z_b ||_2^2 + \sum_{(i,j)}|| \nabla z(i,j)||_2^2$$

$f$ est la fonction coût à minimiser par rapport à l'image bruitée $z_b$.

Les variables de décisions sont les coordonnées de la matrice $z$, au nombre de $n \times m$.

Tous les pixels devant en outre être compris entre $0$ et $1$, le vecteur contrainte $c(z)$ contient l'ensemble des coordonnées de $z$ avec $$\forall (i,j) \in \mathbb{R}^{n}\times \mathbb{R}^{m} c_{i,j}= u_{i,j}-1 \text{ et } c_{n+i,m+j}=-u_{n+i,m+j}$$
Il y a donc $2n\times 2m=4nm$ contraintes. 

2. f est convexe et différentiable comme la somme et la composition de fonctions quadratiques et différentiables. 

3. Montrons que $$ \nabla f  = u - u_b - 2\text{div}(\nabla u) = u - u_b - 2\Delta u$$

$\red{\text{À  COMPLÉTER}}$